In [30]:
import cv2
import mediapipe as mp
import glob, os
import subprocess

In [31]:
## initialize pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [32]:
fileList = glob.glob('video_source/*' , recursive=True)
fileList.sort()
fileList

['video_source\\IMG_8839.MOV',
 'video_source\\IMG_8840.MOV',
 'video_source\\IMG_8842.MOV',
 'video_source\\IMG_8843.MOV',
 'video_source\\IMG_8844.MOV',
 'video_source\\IMG_8845.MOV',
 'video_source\\IMG_8846.MOV',
 'video_source\\IMG_8848.MOV']

In [33]:
ffmpeg_path = "../../../ffmpeg/bin/ffmpeg.exe"

# TRIM SECONDS FROM BEG OF VIDEO

In [34]:
def trim_video_from_beg(ffmpeg_path,input_video,output_video,shortened_duration):    
 
    command = [ffmpeg_path, '-i', input_video, '-ss', str(shortened_duration), "-c", "copy", output_video]

    try:
        # Run the command
        subprocess.run(command, check=True)
        print(f"Video shortened successfully to {shortened_duration} seconds.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# MERGE TWO VIDEOES SIDE BY SIDE

In [35]:
def merge_two_videos_sbys(ffmpeg_path,input_video1,input_video2,output_video):
    # Get the number of CPU cores
    cpu_cores = os.cpu_count()
    
    # Example command to combine videos side by side with all CPUs
    command = [
        ffmpeg_path,
        '-i', input_video1,
        '-i', input_video2,
        '-filter_complex', f'[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]',
        '-map', '[vid]',
        '-threads', str(cpu_cores),  # Use all available CPU cores
        output_video
    ]

    try:
        # Run the command
        subprocess.run(command, check=True)
        print("Videos combined side by side successfully using all CPUs.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")   

# MAKE GIF FROM VIDEO

In [36]:
def make_gif(input_video,output_gif):

    # Example command to convert video to animated GIF
    command = [ffmpeg_path, '-i', input_video, '-vf', 'fps=10,scale=640:-1', '-c:v', 'gif', output_gif]

    try:
        # Run the command
        subprocess.run(command, check=True)
        print("Video converted to animated GIF successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")


# ADD MOTION TRACK TO VIDEO

In [37]:
def add_motion_track(input_video,output_video):

    # Create a VideoCapture object
    cap = cv2.VideoCapture(input_video)

    # Get the original video's width and height
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Get the frame rate
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(width)
    print(height)
    print(fps)

    # Define the codec and create a VideoWriter object
    # Set the video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #fourcc = cv2.VideoWriter_fourcc(*'MP4V') 
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))  # You may need to adjust the resolution (640x480 in this example)

    # Initialize the pose detection module
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    while cap.isOpened():
        # Read a frame
        ret, frame = cap.read()

        if not ret:
            break

        try:
            # Resize the frame for portrait video
            #frame = cv2.resize(frame, (350, 600))

            # Convert to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the frame for pose detection
            pose_results = pose.process(frame_rgb)

            # Draw skeleton on the frame
            #mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            #Draw skeleton on the frame excluding eyes
            mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))

            # Extract and print name, x, y coordinates of each landmark along with timestamp
            if pose_results.pose_landmarks:
                timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)  # Get the current timestamp in milliseconds
                for idx, landmark in enumerate(pose_results.pose_landmarks.landmark):
                    landmark_name = mp_pose.PoseLandmark(idx).name
                    x = int(landmark.x * width)
                    y = int(landmark.y * height)
                    #print(f"Timestamp: {timestamp} ms, Landmark {landmark_name}: ({x}, {y})")

                 # Add timestamp as text to the frame
                cv2.putText(frame, f'Timestamp: {int(timestamp)} ms', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Write the frame to the output video file
            out.write(frame)

            # Display the frame
            cv2.imshow('Output', frame)

        except Exception as e:
            print(f"Error: {e}")
            break

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) == ord('q'):
            break

    # Release the VideoCapture and VideoWriter objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# RUN JOB

In [38]:
input_video = fileList[1]
output_video = 'shortened_video-A.mov'
shortened_duration = .8
trim_video_from_beg(ffmpeg_path,input_video,output_video,shortened_duration)

input_video = fileList[0]
output_video = 'shortened_video-B.mov'
shortened_duration = 0
trim_video_from_beg(ffmpeg_path,input_video,output_video,shortened_duration)

add_motion_track('shortened_video-A.mov','motionA.avi')
add_motion_track('shortened_video-B.mov','motionB.avi')

# Output video file (combined side by side)
output_video = 'combined_videoC.mp4'
input_video1 = 'motionB.avi'
input_video2 = 'motionA.avi'
merge_two_videos_sbys(ffmpeg_path,input_video1,input_video2,output_video) 

make_gif(output_video,"image.gif")

Video shortened successfully to 0.8 seconds.
Video shortened successfully to 0 seconds.
1080
1920
30.0
1080
1920
30.0
Videos combined side by side successfully using all CPUs.
Video converted to animated GIF successfully.


In [39]:
input_video = fileList[3]
output_video = 'shortened_video-A.mov'
shortened_duration = 0
trim_video_from_beg(ffmpeg_path,input_video,output_video,shortened_duration)

input_video = fileList[2]
output_video = 'shortened_video-B.mov'
shortened_duration = 4.2
trim_video_from_beg(ffmpeg_path,input_video,output_video,shortened_duration)

add_motion_track('shortened_video-A.mov','motionA.avi')
add_motion_track('shortened_video-B.mov','motionB.avi')

# Output video file (combined side by side)
output_video = 'combined_videoC.mp4'
input_video1 = 'motionB.avi'
input_video2 = 'motionA.avi'
merge_two_videos_sbys(ffmpeg_path,input_video1,input_video2,output_video) 

make_gif(output_video,"image1.gif")

In [ ]:
make_gif(output_video,"image1.gif")

# REDUCE VIDEO SIZE

In [ ]:
input_video = "combined_videoC.mp4"
output_video = "compressed_video.mp4"

# FFmpeg command to compress the video with H.265 codec
ffmpeg_command = [
    ffmpeg_path,
    "-i", input_video,
    "-c:v", "libx265",
    output_video
]

# Execute the FFmpeg command
subprocess.run(ffmpeg_command)

# CAPTURE USB CAM

In [ ]:
import cv2
import mediapipe as mp

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Change the argument in VideoCapture to the index of your USB camera
cap = cv2.VideoCapture(1)  # Change the index to match your USB camera

# Set the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))  # You may need to adjust the resolution (640x480 in this example)

while cap.isOpened():
    # read frame
    _, frame = cap.read()
    try:
        # resize the frame for portrait video
        # frame = cv2.resize(frame, (350, 600))
        # convert to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # process the frame for pose detection
        pose_results = pose.process(frame_rgb)
        # print(pose_results.pose_landmarks)
        
        # draw skeleton on the frame
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Write the frame to the video
        out.write(frame)
        
        # display the frame
        cv2.imshow('Output', frame)
    except:
        break
        
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
out.release()  # Release the video writer
cv2.destroyAllWindows()
